In [4]:
from aicsimageio.writers import OmeTiffWriter
from aicsimageio import aics_image as AI
from aicsimageio import AICSImage, imread
import dask
import dask.array as da
import os


In [5]:
# Path to TIFF file
tif_file_path = './../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/test/SCC_T37C_T1_Scan1_[10838,39481]_component_data.tif'

In [6]:
# this correctly stacks images per tile, no correct metadata

# Paths to input and output directories
input_dir = './../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/test/'
output_file = './../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/stacked_output.ome.tiff'

# Make list of all files with a ".tif" extension in the input directory
input_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]
# Sort the list of files alphabetically
input_files = sorted(input_files)

# Load the first image to determine the shape
first_img = AICSImage(input_files[0]).get_image_dask_data("CZYX")

# Number of tiles (one file per tile)
num_tiles = len(input_files)
# Number of channels in each tile (assumed to be the same for all tiles)
num_channels = first_img.shape[0]

# Initialize an empty dask array to hold all images
# Shape: (Tiles, Channels, Z, Y, X)
stack_shape = (num_tiles, num_channels, first_img.shape[1], first_img.shape[2], first_img.shape[3])
out = da.empty(stack_shape, dtype=first_img.dtype)

# Loop through each file and stack the images
for idx, file in enumerate(input_files):
    img = AICSImage(file).get_image_dask_data("CZYX")
    out[idx] = img

# Reshape the array to match the required order TCZYX (Tiles as time points)
out = out.reshape((num_tiles * num_channels, 1, first_img.shape[1], first_img.shape[2], first_img.shape[3]))

# Save the stacked image using OmeTiffWriter
OmeTiffWriter.save(out, output_file, dim_order="TCZYX")

print(f"Stacked image saved to {output_file}")


Stacked image saved to ./../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/stacked_output.ome.tiff


In [7]:
#this incorrectly stacks images per channel, no correct metadata
# Define the number of channels
num_channels = 6

# Paths to input and output directories
input_dir = './../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/test/'
output_file = './../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/stacked_output.ome.tiff'

# Make list of all files with a ".tif" extension in the input directory
input_files = [os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith('.tif')]
# Sort the list of files alphabetically
input_files = sorted(input_files)

# Load the first image to determine the shape
first_img = AICSImage(input_files[0]).get_image_dask_data("CZYX")

# Initialize an empty dask array to hold all images
stack_shape = (first_img.shape[0], len(input_files), first_img.shape[1], first_img.shape[2], first_img.shape[3])
out = da.empty(stack_shape, dtype=first_img.dtype)

# Loop through each file and stack the images along the channel axis
for idx, file in enumerate(input_files):
    img = AICSImage(file).get_image_dask_data("CZYX")
    out[:, idx] = img

# Save the stacked image using OmeTiffWriter
OmeTiffWriter.save(out, output_file, dim_order="TCZYX")

print(f"Stacked image saved to {output_file}")


Stacked image saved to ./../data/Human_squamous_cell_carcinoma_stained_with_SignalStar_mIHC_technology/stacked_output.ome.tiff
